In [1]:
from pprint import pprint
import yaml

from data_algebra.data_ops import *
import data_algebra.env
import data_algebra.yaml
import data_algebra.PostgreSQL


db_model = data_algebra.PostgreSQL.PostgreSQLModel()

data_algebra.yaml.fix_ordered_dict_yaml_rep()
data_algebra.env.push_onto_namespace_stack(locals())

t1 = TableDescription('t1', ['x', 'y'])

In [2]:
print(t1)

TableDescription(table_name='t1', column_names=['x', 'y'])


In [3]:
print(t1.to_sql(db_model, pretty=True))


SELECT "x",
       "y"
FROM "t1"


In [4]:
ops = t1 . extend({'v':'x + 1'})

In [5]:
print(ops)

TableDescription(table_name='t1', column_names=['x', 'y']) .\
   extend({'v': 'x + 1'})


In [6]:
print(ops.to_sql(db_model, pretty=True))

SELECT "x",
       "y",
       "x" + 1 AS "v"
FROM
  (SELECT "x",
          "y"
   FROM "t1") "sq_0"


In [7]:
opse = t1 . extend({'y':'y.max()'}, partition_by = ['x'])
print(opse)
print(opse.to_sql(db_model, pretty=True))

TableDescription(table_name='t1', column_names=['x', 'y']) .\
   extend({'y': 'y.max()'}, partition_by=['x'])
SELECT "x",
       MAX("y") OVER (PARTITION BY "x") AS "y"
FROM
  (SELECT "x",
          "y"
   FROM "t1") "sq_0"


In [8]:
t1b = TableDescription('t1', ['x', 'y', 'one_more'])
t2 = TableDescription('t2', ['x', 'z'])

ops = ( 
        t1 . 
            extend({'x':'x + 1'}) . 
            natural_join(
                b=t1b,
                by=['x', 'y']) . 
            natural_join(
                b=(t2 . extend({'f':'x + 1'})),
                by='x') 
)

try:
    print(ops.get_tables())
except Exception as ex:
    print(ex)

Two tables with key t1 have different column sets.


In [9]:
t1b = TableDescription('t1', ['x', 'y'])
t2 = TableDescription('t2', ['x', 'z'])

ops = ( 
        t1 . 
            extend({'x':'x + 1'}) . 
            natural_join(
                b=t1b,
                by=['x', 'y']) . 
            natural_join(
                b=(t2 . extend({'f':'x + 1'})),
                by='x') 
)
print(ops)

TableDescription(table_name='t1', column_names=['x', 'y']) .\
   extend({'x': 'x + 1'}) .\
   natural_join(b=
      TableDescription(table_name='t1', column_names=['x', 'y']),
      by=['x', 'y'], jointype='INNER') .\
   natural_join(b=
      TableDescription(table_name='t2', column_names=['x', 'z']) .\
         extend({'f': 'x + 1'}),
      by=['x'], jointype='INNER')


In [10]:
print(ops.get_tables())

{'t1': TableDescription(table_name='t1', column_names=['x', 'y']), 't2': TableDescription(table_name='t2', column_names=['x', 'z'])}


In [11]:
print(ops.columns_used())

{'t1': {'x', 'y'}, 't2': {'x', 'z'}}


In [12]:
print(ops.to_sql(db_model, pretty=True))

SELECT COALESCE("lq_4"."x", "rq_5"."x") AS "x",
       "y",
       "z",
       "f"
FROM
  (SELECT COALESCE("lq_1"."x", "rq_2"."x") AS "x",
          COALESCE("lq_1"."y", "rq_2"."y") AS "y"
   FROM
     (SELECT "y",
             "x" + 1 AS "x"
      FROM
        (SELECT "x",
                "y"
         FROM "t1") "sq_0") "lq_1"
   INNER JOIN
     (SELECT "x",
             "y"
      FROM "t1") "rq_2") "lq_4"
INNER JOIN
  (SELECT "x",
          "z",
          "x" + 1 AS "f"
   FROM
     (SELECT "x",
             "z"
      FROM "t2") "sq_3") "rq_5"


In [13]:
p = ops.collect_representation()
pprint(p)

[OrderedDict([('op', 'TableDescription'),
              ('table_name', 't1'),
              ('qualifiers', {}),
              ('column_names', ['x', 'y']),
              ('key', 't1')]),
 OrderedDict([('op', 'Extend'),
              ('ops', {'x': 'x + 1'}),
              ('partition_by', []),
              ('order_by', []),
              ('reverse', [])]),
 OrderedDict([('op', 'NaturalJoin'),
              ('by', ['x', 'y']),
              ('jointype', 'INNER'),
              ('b',
               [OrderedDict([('op', 'TableDescription'),
                             ('table_name', 't1'),
                             ('qualifiers', {}),
                             ('column_names', ['x', 'y']),
                             ('key', 't1')])])]),
 OrderedDict([('op', 'NaturalJoin'),
              ('by', ['x']),
              ('jointype', 'INNER'),
              ('b',
               [OrderedDict([('op', 'TableDescription'),
                             ('table_name', 't2'),
                

In [14]:

dmp = yaml.dump(p)
print(dmp)

- op: TableDescription
  table_name: t1
  qualifiers: {}
  column_names:
  - x
  - y
  key: t1
- op: Extend
  ops:
    x: x + 1
  partition_by: []
  order_by: []
  reverse: []
- op: NaturalJoin
  by:
  - x
  - y
  jointype: INNER
  b:
  - op: TableDescription
    table_name: t1
    qualifiers: {}
    column_names:
    - x
    - y
    key: t1
- op: NaturalJoin
  by:
  - x
  jointype: INNER
  b:
  - op: TableDescription
    table_name: t2
    qualifiers: {}
    column_names:
    - x
    - z
    key: t2
  - op: Extend
    ops:
      f: x + 1
    partition_by: []
    order_by: []
    reverse: []



In [15]:
ops_back = data_algebra.yaml.to_pipeline(yaml.safe_load(dmp))
print(ops_back)



TableDescription(table_name='t1', column_names=['x', 'y']) .\
   extend({'x': 'x + 1'}) .\
   natural_join(b=
      TableDescription(table_name='t1', column_names=['x', 'y']),
      by=['x', 'y'], jointype='INNER') .\
   natural_join(b=
      TableDescription(table_name='t2', column_names=['x', 'z']) .\
         extend({'f': 'x + 1'}),
      by=['x'], jointype='INNER')


In [16]:
print(ops_back.to_python(pretty=True))


TableDescription(table_name="t1", column_names=["x", "y"]).extend(
    {"x": "x + 1"}
).natural_join(
    b=TableDescription(table_name="t1", column_names=["x", "y"]),
    by=["x", "y"],
    jointype="INNER",
).natural_join(
    b=TableDescription(table_name="t2", column_names=["x", "z"]).extend({"f": "x + 1"}),
    by=["x"],
    jointype="INNER",
)

